In [1]:
import autogluon
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor

In [2]:

df = pd.read_csv('train.csv')
train = df.sample(frac=0.7)
test = df.drop(df.sample(frac=0.7).index)

In [3]:
label = 'SalePrice'#定义Lable，即目标变量
save_path = 'agModels-predictClass'#模型保存路径

#训练
predictor = TabularPredictor(label=label, path=save_path).fit(train,presets='best_quality')
#'best_quality'：选择精度最高的训练方法，但时间开销会变大

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "agModels-predictClass\"
AutoGluon Version:  0.8.2
Python Version:     3.9.12
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.22621
Disk Space Avail:   107.75 GB / 214.75 GB (50.2%)
Train Data Rows:    1022
Train Data Columns: 80
Label Column: SalePrice
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == int and many unique label-values observed).
	Label info (max, min, mean, stddev): (625000, 34900, 183863.50391, 79151.79019)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeature

	0.28s	 = Validation runtime
Fitting model: XGBoost_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-25736.674	 = Validation score   (-root_mean_squared_error)
	3.31s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: NeuralNetTorch_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-25744.6557	 = Validation score   (-root_mean_squared_error)
	17.58s	 = Training   runtime
	0.46s	 = Validation runtime
Fitting model: LightGBMLarge_BAG_L2 ...
	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy
	-26975.3762	 = Validation score   (-root_mean_squared_error)
	8.86s	 = Training   runtime
	0.14s	 = Validation runtime
Fitting model: WeightedEnsemble_L3 ...
	-24075.9945	 = Validation score   (-root_mean_squared_error)
	0.32s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 815.47s ... Best model: "WeightedEn

In [4]:
#定义test
y_test = test[label] 
test_data_nolab = test.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

#从save_path导入训练好的模型
# unnecessary, just demonstrates how to load previously-trained predictor from file
predictor = TabularPredictor.load(save_path)  

#预测
y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)

#查看效果
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Evaluation: root_mean_squared_error on test data: -20612.87680728128
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -20612.87680728128,
    "mean_squared_error": -424890690.2721545,
    "mean_absolute_error": -9849.510791238585,
    "r2": 0.9302920400343131,
    "pearsonr": 0.9665465734203362,
    "median_absolute_error": -5742.96484375
}


Predictions:  
 11      364274.500000
12      139134.500000
14      153626.000000
16      151164.359375
17      101810.687500
            ...      
1438    146999.187500
1443    109734.718750
1446    146039.312500
1448    114818.078125
1449     98581.640625
Name: SalePrice, Length: 438, dtype: float32


In [5]:
#使用训练好的模型，调用.leaderboard，查看在test上的效果
predictor.leaderboard(test, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMLarge_BAG_L1,-16992.586516,-28518.874084,1.001252,0.385166,50.733941,1.001252,0.385166,50.733941,1,True,11
1,XGBoost_BAG_L2,-17610.543381,-25736.673993,4.983896,3.499542,542.245422,0.407594,0.142745,3.306458,2,True,19
2,XGBoost_BAG_L1,-17725.343281,-26681.863897,0.371492,0.173992,10.466781,0.371492,0.173992,10.466781,1,True,9
3,RandomForestMSE_BAG_L2,-17971.977043,-25795.147916,4.703294,3.487312,540.739406,0.126992,0.130515,1.800441,2,True,15
4,ExtraTreesMSE_BAG_L2,-18614.300150,-24746.408338,4.761641,3.520808,539.641477,0.185339,0.164011,0.702512,2,True,17
5,LightGBMLarge_BAG_L2,-19060.045137,-26975.376214,4.839608,3.499504,547.795669,0.263305,0.142706,8.856705,2,True,21
6,RandomForestMSE_BAG_L1,-19336.135361,-29528.333482,0.130158,0.136418,1.839291,0.130158,0.136418,1.839291,1,True,5
7,ExtraTreesMSE_BAG_L1,-19496.515349,-27976.860894,0.209007,0.128584,0.640530,0.209007,0.128584,0.640530,1,True,7
8,WeightedEnsemble_L3,-19688.643007,-24075.994538,6.610181,4.624104,719.513921,0.011930,0.000000,0.318033,3,True,22
9,CatBoost_BAG_L2,-19905.136097,-24900.892317,4.776093,3.572643,691.694491,0.199791,0.215845,152.755526,2,True,16


In [8]:
pre_data = pd.read_csv('test.csv')
predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(pre_data)

result = pd.DataFrame(pre_data['Id'])
result['SalePrice'] = y_pred

#结果保存为submission.csv文件
result.to_csv('submission.csv',index=False)